#Flower Classifier with MediaPipe Model Maker
Margaret Maynard-Reid, 3/31/2023


Reference: https://developers.google.com/mediapipe/solutions/vision/image_classifier/customize

* Data input: Flower photos
* Model output: flower-model.tflite

In [ ]:
!python --version
!pip install --upgrade pip
!pip install mediapipe-model-maker

## Imports

In [2]:
from google.colab import files
import os
import tensorflow as tf
assert tf.__version__.startswith('2')

from mediapipe_model_maker import image_classifier

import matplotlib.pyplot as plt

/usr/local/lib/python3.9/dist-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.9.0 and strictly below 2.12.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.12.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


In [3]:
print(tf. __version__)

2.12.0


## Flower Data

In [4]:
image_path = tf.keras.utils.get_file(
    'flower_photos.tgz',
    'https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz',
    extract=True)
image_path = os.path.join(os.path.dirname(image_path), 'flower_photos')

228813984/228813984 [==============================] - 10s 0us/step


In [5]:
print(image_path)
labels = []
for i in os.listdir(image_path):
  if os.path.isdir(os.path.join(image_path, i)):
    labels.append(i)
print(labels)

/root/.keras/datasets/flower_photos
['roses', 'daisy', 'tulips', 'sunflowers', 'dandelion']


In [6]:
NUM_EXAMPLES = 5

for label in labels:
  label_dir = os.path.join(image_path, label)
  example_filenames = os.listdir(label_dir)[:NUM_EXAMPLES]
  fig, axs = plt.subplots(1, NUM_EXAMPLES, figsize=(10,2))
  for i in range(NUM_EXAMPLES):
    axs[i].imshow(plt.imread(os.path.join(label_dir, example_filenames[i])))
    axs[i].get_xaxis().set_visible(False)
    axs[i].get_yaxis().set_visible(False)
  fig.suptitle(f'Showing {NUM_EXAMPLES} examples for {label}')

plt.show()

In [7]:
data = image_classifier.Dataset.from_folder(image_path)
train_data, remaining_data = data.split(0.8)
test_data, validation_data = remaining_data.split(0.5)

## Train Image Classifier

In [8]:
spec = image_classifier.SupportedModels.MOBILENET_V2
hparams = image_classifier.HParams(export_dir="exported_model")
options = image_classifier.ImageClassifierOptions(supported_model=spec, hparams=hparams)

In [9]:
model = image_classifier.ImageClassifier.create(
    train_data = train_data,
    validation_data = validation_data,
    options=options,
)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1280)              2257984   
                                                                 
 dropout (Dropout)           (None, 1280)              0         
                                                                 
 dense (Dense)               (None, 5)                 6405      
                                                                 
Total params: 2,264,389
Trainable params: 6,405
Non-trainable params: 2,257,984
_________________________________________________________________
None
Epoch 1/10
1468/1468 [==============================] - 35s 16ms/step - loss: 1.8267 - accuracy: 0.2411 - val_loss: 1.3530 - val_accuracy: 0.5177
Epoch 2/10
1468/1468 [==============================] - 22s 15ms/step - loss: 1.2387 - accuracy: 0.5616 - val_loss: 0.9087 - val_accuracy: 0.7766
Epoch 3/1

In [10]:
loss, acc = model.evaluate(test_data)
print(f'Test loss:{loss}, Test accuracy:{acc}')

12/12 [==============================] - 4s 94ms/step - loss: 0.6737 - accuracy: 0.8801
Test loss:0.6736714839935303, Test accuracy:0.8801090121269226


## Export and Download .tflite Model

In [11]:
model.export_model(model_name="flower-model.tflite")

In [12]:
!ls exported_model
files.download('exported_model/flower-model.tflite')

checkpoint  flower-model.tflite  metadata.json	summaries


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>